In [1]:
import warnings
import keplergl
import numpy as np
import osmnx as ox
import pandas as pd
import geopandas as gpd
import plotly.express as px
from skgstat import Variogram
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pykrige.ok import OrdinaryKriging
from scipy.interpolate import NearestNDInterpolator
from tobler.area_weighted import area_interpolate
import plotly.express as px
import json
import ogr
import ipywidgets as widgets
from IPython.display import display

In [2]:
# this isn't really a .geojson, it's just a json list of features
with open('van_poly.geojson') as f:
  blocks = json.load(f)

In [3]:
# we need to convert it to geojson format by putting back in this dictionary structure
geo_json = {"type": "FeatureCollection", "features": blocks}
geo_json["features"][0]

{'type': 'Feature',
 'properties': {'DBUID': '59150244005',
  'DBRPLAMX': 4030983.593945,
  'DBRPLAMY': 2005717.42964501,
  'PRUID': '59',
  'PRNAME': 'British Columbia / Colombie-Britannique',
  'CDUID': '5915',
  'CDNAME': 'Greater Vancouver',
  'CDTYPE': 'RD',
  'CCSUID': '5915020',
  'CCSNAME': 'Greater Vancouver A',
  'CSDUID': '5915046',
  'CSDNAME': 'North Vancouver',
  'CSDTYPE': 'DM',
  'ERUID': '5920',
  'ERNAME': 'Lower Mainland--Southwest / Lower Mainland--Sud-ouest',
  'FEDUID': '59002',
  'FEDNAME': 'Burnaby North--Seymour / Burnaby-Nord--Seymour',
  'SACCODE': '933',
  'SACTYPE': '1',
  'CMAUID': '933',
  'CMAPUID': '59933',
  'CMANAME': 'Vancouver',
  'CMATYPE': 'B',
  'CTUID': '9330110.03',
  'CTNAME': '0110.03',
  'ADAUID': '59150014',
  'DAUID': '59150244'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-122.96251616532064, 49.32169736741242],
    [-122.96222837991496, 49.321261359289615],
    [-122.9621961748921, 49.32116586646329],
    [-122.96234696135333, 

In [4]:
df = gpd.GeoDataFrame.from_features(geo_json)
df = df[['geometry', 'DBUID']]
df.head()

,geometry,DBUID
0,"POLYGON ((-122.96252 49.32170, -122.96223 49.3...",59150244005
1,"POLYGON ((-122.96832 49.34238, -122.96780 49.3...",59150244008
2,"POLYGON ((-123.07798 49.25263, -123.07799 49.2...",59150372006
3,"POLYGON ((-123.07991 49.25206, -123.07802 49.2...",59150372007
4,"POLYGON ((-123.08188 49.25090, -123.08189 49.2...",59150373001


In [5]:
df["DBUID"] = pd.to_numeric(df["DBUID"])

In [7]:
near = pd.read_csv('ttm6_719.csv')

near = near[['fromId', 'type', 't1', 'avg_time']]
near.columns = ['DBUID', 'type', 'time', 'avg_time']

# near = near[['fromId', 'type', 't1', 'avg_time', 'Dest', 'Latitude', 'Longitude']]
# near.columns = ['DBUID', 'type', 'time', 'avg_time', 'Destination', 'Latitude', 'Longitude']
near.head()

,DBUID,type,time,avg_time
0,59150004004,gallery,07:00,38
1,59150004004,gallery,08:00,37
2,59150004004,gallery,09:00,35
3,59150004004,gallery,10:00,35
4,59150004004,gallery,11:00,35


In [ ]:
## nearest for peak time

near = pd.read_csv('Nearest1_peak.csv')

near = near[['fromId', 'type', 't1', 'avg_time', 'Dest']]
near.columns = ['DBUID', 'type', 'time', 'avg_time', 'Destination']
near.head()

In [8]:
near1 = df.merge(near, on='DBUID')
near1.head()
# near1['geometry'].nunique()

,geometry,DBUID,type,time,avg_time
0,"POLYGON ((-122.96252 49.32170, -122.96223 49.3...",59150244005,gallery,07:00,35
1,"POLYGON ((-122.96252 49.32170, -122.96223 49.3...",59150244005,gallery,08:00,34
2,"POLYGON ((-122.96252 49.32170, -122.96223 49.3...",59150244005,gallery,09:00,37
3,"POLYGON ((-122.96252 49.32170, -122.96223 49.3...",59150244005,gallery,10:00,31
4,"POLYGON ((-122.96252 49.32170, -122.96223 49.3...",59150244005,gallery,11:00,38


In [8]:
# Save for cloud storage - Kepler.gl

near1.to_csv('near1_sat.csv', index=False)

In [ ]:
near1_719 = keplergl.KeplerGl(height=500)
near1_719.add_data(data=near1.copy(), name="Nearest Amenity")
near1_719

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [10]:
near1_719.save_to_html(file_name='Nearest_719_Kepler.html')

Map saved to Nearest_Peak_Kepler.html!
